In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '0'


In [2]:
from typing import *
import pandas as pd
import os
import torch.nn as nn
import torch
from transformers import LongformerTokenizerFast, LongformerForSequenceClassification, Trainer, TrainingArguments, LongformerConfig, T5Config
from datasets import Dataset
from torch.utils.data import DataLoader
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import wandb
import argparse
from functools import partial

from helpers import find_next_dir_index , explode_train_target, convert_to_list, merge_embs_to_seq, compute_metrics_LSTM, group_train_val
from MyTrainer import MyTrainer 
from BoXHED_Fuse.models.ClinicalLSTM import ClinicalLSTM
# from ..models.ClinicalLSTM import ClinicalLSTM



# ===== Initialize Args ===== 
# parser = argparse.ArgumentParser()
# parser.add_argument('--test', action='store_true', help='enable testing mode')
# parser.add_argument('--use-wandb', action = 'store_true', help = 'enable wandb', default=False)
# parser.add_argument('--gpu-no', dest = 'GPU_NO', help='use GPU_NO specified (this may be a single number or several. eg: 1 or 1,2,3,4)')
# parser.add_argument('--note-type', dest = 'note_type', help='which notes, radiology or discharge?')
# parser.add_argument('--num-epochs', dest = 'num_epochs', help = 'num_epochs to train')
# parser.add_argument('--noteid-mode', dest = 'noteid_mode', help = 'kw: all or recent')
# args = parser.parse_args()

args = argparse.Namespace
args.test = True
args.GPU_NO = -1
args.note_type = 'radiology'
args.num_epochs = 1
args.noteid_mode = 'all'
args.model_name = 'LSTM'

args.num_epochs = int(args.num_epochs)

model_name_ft1 = 'Clinical-T5-Base'
train_embs_path = f'{os.getenv("BHF_ROOT")}/JSS_SUBMISSION_NEW/data/embs{"/testing" if args.test else ""}/{model_name_ft1}_{args.note_type[:3]}_{args.noteid_mode}_out/from_epoch1/10/train_embs.pt'
train_target_path = f'{os.getenv("BHF_ROOT")}/JSS_SUBMISSION_NEW/data/targets{"/testing" if args.test else ""}/till_end_mimic_iv_extra_features_train_NOTE_TARGET_2_{args.note_type[:3]}_{args.noteid_mode}.csv'
model_name = 'clinical_lstm'
model_out_dir = f'{os.getenv("BHF_ROOT")}/model_outputs/{model_name}_{args.note_type[:3]}_{args.noteid_mode}_out'


In [3]:
def validate_train_embseq(train_embseq, train_target):
    train_embseq = train_embseq.copy()
    train_embseq['emb_seq_len'] = train_embseq['emb_seq'].apply(len)
    train_target['NOTE_ID_SEQ_len'] = train_target['NOTE_ID_SEQ'].apply(len)
    assert((train_target['NOTE_ID_SEQ_len'] == train_embseq['emb_seq_len']).all())


In [4]:

if not os.path.exists(model_out_dir):
    os.makedirs(model_out_dir)
run_cntr = find_next_dir_index(model_out_dir)
model_out_dir = os.path.join(model_out_dir, str(run_cntr))
assert(not os.path.exists(model_out_dir))
# os.makedirs(model_out_dir) # FIXME
print(f'created all dirs in model_out_dir', model_out_dir)

if args.test:
    # train_target_path = os.path.join(os.path.dirname(train_target_path), 'testing', os.path.basename(train_target_path))
    model_out_dir = os.path.join(os.path.dirname(model_out_dir), 'testing', os.path.basename(model_out_dir))


# ===== Read Data =====
train_embs = torch.load(train_embs_path)
train_target = pd.read_csv(train_target_path, converters = {'NOTE_ID_SEQ': convert_to_list})
target = 'delta_in_2_days' # FIXME
train_target.rename(columns = {target:'label'}, inplace=True)
# ===== Merge data into {note_embs_seq, label}, where note_embs_seq is a list of embs =====
train_target_exploded = explode_train_target(train_target)
train_embs_df = pd.DataFrame()
train_embs_df['emb'] = [np.array(e) for e in train_embs]
train_embs_df = pd.concat([train_target_exploded, train_embs_df], axis=1)
train_embseq = merge_embs_to_seq(train_target, train_embs_df)


created all dirs in model_out_dir /home/ugrads/a/aa_ron_su/BoXHED_Fuse/BoXHED_Fuse//model_outputs/clinical_lstm_rad_all_out/21


In [43]:
output.shape

torch.Size([32, 2, 64])

In [25]:
train_embseq

,ICUSTAY_ID,NOTE_ID,label,NOTE_ID_SEQ,emb_seq
0,30676350.0,10003019-RR-66,0,"[10003019-RR-16, 10003019-RR-17, 10003019-RR-1...","[[0.04571784197735888, 0.02954959064224078, -0..."
1,31090461.0,10002155-RR-52,0,"[10002155-RR-22, 10002155-RR-23, 10002155-RR-2...","[[0.040107877819736744, 0.07095444332914923, 0..."
2,31205490.0,10001725-RR-12,0,"[10001725-RR-10, 10001725-RR-11, 10001725-RR-12]","[[0.007828903479476993, 0.08233785253433663, 0..."
3,32128372.0,10003400-RR-66,0,"[10003400-RR-49, 10003400-RR-50, 10003400-RR-5...","[[0.022228072317655084, 0.09650489123149635, 0..."
4,32128372.0,10003400-RR-67,0,"[10003400-RR-49, 10003400-RR-50, 10003400-RR-5...","[[0.022228072317655084, 0.09650489123149635, 0..."
5,32128372.0,10003400-RR-68,0,"[10003400-RR-49, 10003400-RR-50, 10003400-RR-5...","[[0.022228072317655084, 0.09650489123149635, 0..."
6,33685454.0,10002155-RR-29,0,"[10002155-RR-22, 10002155-RR-23, 10002155-RR-2...","[[0.040107877819736744, 0.07095444332914923, 0..."
7,33685454.0,10002155-RR-31,0,"[10002155-RR-22, 10002155-RR-23, 10002155-RR-2...","[[0.040107877819736744, 0.07095444332914923, 0..."
8,33685454.0,10002155-RR-32,0,"[10002155-RR-22, 10002155-RR-23, 10002155-RR-2...","[[0.040107877819736744, 0.07095444332914923, 0..."
9,33685454.0,10002155-RR-33,0,"[10002155-RR-22, 10002155-RR-23, 10002155-RR-2...","[[0.040107877819736744, 0.07095444332914923, 0..."


In [5]:
# ===== Validate =====

validate_train_embseq(train_embseq, train_target)

In [6]:
from torch.utils import data

# maximum sequence length
max_num_notes = 32
doc_emb_size = 64 # 768
    
class Sequential_Dataset(data.Dataset):

    def __init__(self, df):
        'Initialization'
        self.df = df

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.df)

    def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        # Load data and get label        
        y = self.df['label'].iloc[index]
        emb_seq = torch.tensor(self.df['emb_seq'].iloc[index])
        emb_seq_out = torch.zeros(size=(max_num_notes, doc_emb_size), dtype=torch.float)  
      
        if len(emb_seq) > max_num_notes:
            emb_seq_out[:max_num_notes] = emb_seq[:max_num_notes]
        else:
            emb_seq_out[:len(emb_seq)] = emb_seq

        return emb_seq_out.cuda(), y

In [7]:
def test_finetune(data_generator, model):
    '''
    tests finetuned model
    '''
    y_pred = []
    y_label = []
    model.eval()
    loss_accumulate = 0.0
    count = 0.0
    for i, (output, label) in enumerate(data_generator):
        output = output.permute(1,0,2)
        score = model(output)
        
        m = torch.nn.Sigmoid()
        logits = torch.squeeze(m(score))
        loss_fct = torch.nn.BCELoss()            
        
        label = torch.from_numpy(np.array(label)).float().cuda()

        loss = loss_fct(logits, label)
        
        loss_accumulate += loss
        count += 1
        
        logits = logits.detach().cpu().numpy()
        
        label_ids = label.to('cpu').numpy()
        y_label = y_label + label_ids.flatten().tolist()
        y_pred = y_pred + logits.flatten().tolist()
        outputs = np.asarray([1 if i else 0 for i in (np.asarray(y_pred) >= 0.5)])
    loss = loss_accumulate/count

    # return y_label, y_pred  # DEBUG
    metrics = compute_metrics_LSTM(y_label, y_pred)
    return metrics, y_pred, loss.item()

In [35]:
import copy

def main_finetune(train_embseq, lr, batch_size, train_epoch):
    '''
    finetune the model. Here, we are finetuning the LSTM!
    
    
    '''

    lr = lr
    BATCH_SIZE = batch_size
    train_epoch = train_epoch
    
    loss_history = []
    
    model = ClinicalLSTM()
    model.cuda()
    
    if torch.cuda.device_count() > 1:
        print("Let's use", torch.cuda.device_count(), "GPUs!")
        model = nn.DataParallel(model, dim = 0)
            
    print('--- Data Preparation ---')
    
    params = {'batch_size': BATCH_SIZE,
              'shuffle': True,
              'num_workers': 0, 
              'drop_last': True}
    
    train_idxs, val_idxs = group_train_val(train_embseq['ICUSTAY_ID'])
    train_data = train_embseq.iloc[train_idxs]
    val_data = train_embseq.iloc[val_idxs]
    train_data = train_data[['emb_seq', 'label']]
    val_data = val_data[['emb_seq', 'label']]



    training_set = Sequential_Dataset(train_data)
    training_generator = data.DataLoader(training_set, **params)

    validation_set = Sequential_Dataset(val_data)
    validation_generator = data.DataLoader(validation_set, **params)
    
    
    opt = torch.optim.Adam(model.parameters(), lr = lr)
    # early stopping
    max_auc = 0
    model_max = copy.deepcopy(model)
   
    print('--- Go for Training ---')
    torch.backends.cudnn.benchmark = True
    for epo in range(train_epoch):
        model.train()
        for i, (output, label) in enumerate(training_generator):
            print('batch', i)
            output = output.permute(1,0,2)
            score = model(output.cuda())
       
            label = torch.from_numpy(np.array(label)).float().cuda()
            
            loss_fct = torch.nn.BCELoss()
            m = torch.nn.Sigmoid()
            n = torch.squeeze(m(score))
            
            loss = loss_fct(n, label)
            loss_history.append(loss)
            
            opt.zero_grad()
            loss.backward()
            opt.step()
           
        # every epoch test
        # with torch.set_grad_enabled(False):
        #     # return test_finetune(validation_generator, model)
        #     metrics, logits, loss = test_finetune(validation_generator, model)
        #     if metrics['auc']:
        #         if metrics['auc'] > max_auc:
        #             model_max = copy.deepcopy(model)
        #             max_auc = metrics['auc']
                 
        #     print('Validation at Epoch '+ str(epo + 1) + ' , AUC: '+ str(metrics['auc']) + ' , AUPRC: ' + str(metrics['auprc']))
    
    # print('--- Go for Testing ---')
    # try:
    #     with torch.set_grad_enabled(False):
            
    #         auc, auprc, logits, loss = test_finetune(testing_generator, model_max)
    #         print('Testing AUROC: ' + str(auc) + ' , AUPRC: ' + str(auprc) + ' , Test loss: '+str(loss))
    # except:
    #     print('testing failed')
    return model_max, loss_history

In [59]:
d = train_data.to_dict()

In [65]:
d['emb_seq'][0][0].shape

(64,)

In [ ]:
train_data = Dataset.from_pandas(train_data)


In [89]:
train_data.set_format('torch', columns=['label', 'emb_seq'])

In [81]:
train_data['label'][0]

tensor(0)

In [88]:
train_data['emb_seq'][0].shape

torch.Size([45, 64])

In [56]:
train_embseq.NOTE_ID_SEQ

0     [10003019-RR-16, 10003019-RR-17, 10003019-RR-1...
1     [10002155-RR-22, 10002155-RR-23, 10002155-RR-2...
2      [10001725-RR-10, 10001725-RR-11, 10001725-RR-12]
3     [10003400-RR-49, 10003400-RR-50, 10003400-RR-5...
4     [10003400-RR-49, 10003400-RR-50, 10003400-RR-5...
5     [10003400-RR-49, 10003400-RR-50, 10003400-RR-5...
6     [10002155-RR-22, 10002155-RR-23, 10002155-RR-2...
7     [10002155-RR-22, 10002155-RR-23, 10002155-RR-2...
8     [10002155-RR-22, 10002155-RR-23, 10002155-RR-2...
9     [10002155-RR-22, 10002155-RR-23, 10002155-RR-2...
10    [10002155-RR-22, 10002155-RR-23, 10002155-RR-2...
11    [10001217-RR-9, 10001217-RR-10, 10001217-RR-11...
12    [10002428-RR-48, 10002428-RR-50, 10002428-RR-5...
13                                      [10003046-RR-6]
14                       [10003046-RR-6, 10003046-RR-7]
15        [10003046-RR-6, 10003046-RR-7, 10003046-RR-8]
16    [10003046-RR-6, 10003046-RR-7, 10003046-RR-8, ...
17    [10003046-RR-6, 10003046-RR-7, 10003046-RR

In [39]:
model = ClinicalLSTM()
model.cuda()
score = model(output.cuda())

In [41]:
model

ClinicalLSTM(
  (encoder): LSTM(64, 64, num_layers=2, bidirectional=True)
  (decoder): Sequential(
    (0): Dropout(p=0.1, inplace=False)
    (1): Linear(in_features=128, out_features=32, bias=True)
    (2): ReLU(inplace=True)
    (3): Linear(in_features=32, out_features=1, bias=True)
  )
)

In [40]:
score

tensor([[0.0191],
        [0.0084]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [9]:
# train_data
# training_generator

In [10]:
# train_data

In [11]:
# train_data['emb_seq'].shape

In [12]:
# torch.tensor(df['emb_seq'].iloc[0])

In [13]:
# torch.cuda.device_count()

In [14]:
lr = 1e-5
BATCH_SIZE = 2
train_epoch = 1

loss_history = []

model = ClinicalLSTM()
model.cuda()

if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model, dim = 0)

params = {'batch_size': BATCH_SIZE,
            'shuffle': True,
            'num_workers': 0, 
            'drop_last': True}

train_idxs, val_idxs = group_train_val(train_embseq['ICUSTAY_ID'])
train_data = train_embseq.iloc[train_idxs]
val_data = train_embseq.iloc[val_idxs]
train_data = train_data[['emb_seq', 'label']]
val_data = val_data[['emb_seq', 'label']]



training_set = Sequential_Dataset(train_data)
training_generator = data.DataLoader(training_set, **params)

validation_set = Sequential_Dataset(val_data)
validation_generator = data.DataLoader(validation_set, **params)

opt = torch.optim.Adam(model.parameters(), lr = lr)
# early stopping
# max_auc = 0
# model_max = copy.deepcopy(model)

In [15]:
training_generator.__len__()

9

In [28]:
for i, (output, label) in enumerate(training_generator):
    # print(i)
    # print(label, output)
    output = output.permute(1,0,2)
    score = model(output.cuda())

    label = torch.from_numpy(np.array(label)).float().cuda()

    loss_fct = torch.nn.BCELoss()
    m = torch.nn.Sigmoid()
    n = torch.squeeze(m(score))

    loss = loss_fct(n, label)
    loss_history.append(loss)

    opt.zero_grad()
    loss.backward()
    opt.step()


In [17]:
loss

tensor(0.7570, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)

In [22]:
# ===== Prepare train val split =====

# ===== Train LSTM ===== 

# clin_lstm = ClinicalLSTM()
lr = 1e-5 
batch_size = 2
train_epoch = 1

out = main_finetune(train_embseq, lr, batch_size, train_epoch)



# ===== Save Sequential Embeddings =====
# extract_emb_seq()



--- Data Preparation ---
--- Go for Training ---
batch 0
batch 1
batch 2
batch 3
batch 4
batch 5
batch 6
batch 7
batch 8
Error: Only one class present in y_true. ROC AUC score is not defined in that case.
Validation at Epoch 1 , AUC: None , AUPRC: -0.0


/home/ugrads/a/aa_ron_su/miniconda3/envs/clinical1/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:891: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(


In [45]:
out[0]

ClinicalLSTM(
  (encoder): LSTM(64, 64, num_layers=2, bidirectional=True)
  (decoder): Sequential(
    (0): Dropout(p=0.1, inplace=False)
    (1): Linear(in_features=128, out_features=32, bias=True)
    (2): ReLU(inplace=True)
    (3): Linear(in_features=32, out_features=1, bias=True)
  )
)

In [ ]:
# train_data

In [ ]:
# training_set.df['emb_seq'].iloc[12]

In [ ]:
# precision_recall_fscore_support(out[0], out[1], average='binary')
# accuracy_score(out[0], out[1])
# ValueError: Classification metrics can't handle a mix of binary and continuous targets

In [ ]:
# type(out[0][0]),type(out[1][0])

(float, float)

In [ ]:
train_embseq